## Classificação de textos

### Bibliotecas

In [1]:
#Bibliotecas
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import warnings

#Estilizar conteúdo
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def estilo_tabelas(df, max_altura='300px', casas_decimais=3):
    return (
        df.style.set_table_styles(
            [
                {'selector': 'thead th', 'props': [('font-size', '12px'), ('text-align', 'center'), ('border-bottom', '2px solid #007BFF')]},  # Azul abaixo do nome das colunas
                {'selector': 'td', 'props': [('font-size', '10px'), ('text-align', 'center'), ('max-height', '40px'), ('white-space', 'nowrap'), ('text-overflow', 'ellipsis'), ('overflow', 'hidden'), ('max-width', '100px')]},
                {'selector': 'tr:nth-child(even)', 'props': [('background-color', '#f9f9f9')]},  # Fundo alternado
                {'selector': 'tr:nth-child(odd)', 'props': [('background-color', '#ffffff')]},
                {'selector': 'table', 'props': [('width', '90%'), ('margin-left', 'auto'), ('margin-right', 'auto'), ('border-collapse', 'collapse')]},
                {'selector': 'td, th', 'props': [('border', '1px solid #666')]},  # Bordas cinza escuro
            ]
        ).set_properties(
            **{'background-color': '#f4f4f4', 'border-color': 'darkgray', 'border-style': 'solid', 'border-width': '1px'}
        ).set_table_attributes(
            f'style="height:auto; overflow:auto; max-height:{max_altura}; display:block;"'  
        ).format(
            precision=casas_decimais  
        )
    )

### Base de dados

In [2]:
object_columns = ['Protocolo_S2iD', 'Nome_Municipio', 'Sigla_UF', 'regiao',
                  'Setores Censitários', 'Status', 'DH_Descricao', 'DM_Descricao',
                  'DA_Descricao', 'DA_Polui/cont da água', 'DA_Polui/cont do ar',
                  'DA_Polui/cont do solo', 'DA_Dimi/exauri hídrico',
                  "DA_Incêndi parques/APA's/APP's", 'PEPL_Descricao', 'PEPR_Descricao',
                  'Categoria', 'Grupo', 'Subgrupo', 'Tipo', 'Subtipo']

dtype = {col: 'object' for col in object_columns}

df_eventos = pd.read_csv(
    "https://raw.githubusercontent.com/brunagmoura/PrevisorReconhecimento/refs/heads/main/df_eventos_desastres_rec_nrec.csv",
    sep=';',
    dtype = dtype,
    decimal=',',
)

estilo_tabelas(df_eventos.head(5))

,Protocolo_S2iD,Nome_Municipio,Sigla_UF,regiao,Data_Registro,Data_Evento,codigo_ibge,Setores Censitários,Status,DH_Descricao,DH_MORTOS,DH_FERIDOS,DH_ENFERMOS,DH_DESABRIGADOS,DH_DESALOJADOS,DH_DESAPARECIDOS,DH_OUTROS AFETADOS,DH_total_danos_humanos,DM_Descricao,DM_Uni Habita Danificadas,DM_Uni Habita Destruidas,DM_Uni Habita Valor,DM_Inst Saúde Danificadas,DM_Inst Saúde Destruidas,DM_Inst Saúde Valor,DM_Inst Ensino Danificadas,DM_Inst Ensino Destruidas,DM_Inst Ensino Valor,DM_Inst Serviços Danificadas,DM_Inst Serviços Destruidas,DM_Inst Serviços Valor,DM_Inst Comuni Danificadas,DM_Inst Comuni Destruidas,DM_Inst Comuni Valor,DM_Obras de Infra Danificadas,DM_Obras de Infra Destruidas,DM_Obras de Infra Valor,DM_total_danos_materiais,DA_Descricao,DA_Polui/cont da água,DA_Polui/cont do ar,DA_Polui/cont do solo,DA_Dimi/exauri hídrico,DA_Incêndi parques/APA's/APP's,PEPL_Descricao,PEPL_Assis_méd e emergên(R$),PEPL_Abast de água pot(R$),PEPL_sist de esgotos sanit(R$),PEPL_Sis limp e rec lixo (R$),PEPL_Sis cont pragas (R$),PEPL_distrib energia (R$),PEPL_Telecomunicações (R$),PEPL_Tran loc/reg/l_curso (R$),PEPL_Distrib combustíveis(R$),PEPL_Segurança pública (R$),PEPL_Ensino (R$),PEPL_total_publico,PEPR_Descricao,PEPR_Agricultura (R$),PEPR_Pecuária (R$),PEPR_Indústria (R$),PEPR_Comércio (R$),PEPR_Serviços (R$),PEPR_total_privado,PE_PLePR,Ano_Evento,Empenhado,DensidadePop,Area,Município - UF,PIB,DOMICILIO_AREARURAL,PDEFAGUA,PDEFESGOTO,PDEFLIXO,PDEFSAN,QTDE_FAMILIAS_ATUALIZADAS,Categoria,Grupo,Subgrupo,Tipo,Subtipo,COBRADE,Pop
0,SP-A-3550001-12200-20100101,São Luiz do Paraitinga,SP,Sudeste,2010-01-01,2010-01-01,3550001,nan,Reconhecido,nan,0,0,0,93,4030,0,16,4139,nan,49,20,20643486.932,3.000,0,1121928.638,9.000,1,5235666.976,0,0,0.000,3,3,32411.272,25.000,0.000,65321178.458,92354672.275,nan,nan,nan,nan,nan,nan,nan,7978.159,673157.183,623293.688,2493.175,0.000,330993.880,3739.762,58609.552,0.000,0.000,1087024.191,2787289.589,nan,1642902.433,1149353.560,379785.310,0.000,24433.113,3196474.416,5983764.005,2010,nan,nan,nan,São Luiz do Paraitinga - SP,86448.000,nan,nan,nan,nan,nan,nan,Natural,Hidrológico,Enxurradas,nan,nan,12200,10397
1,SP-A-3518305-11321-20100101,Guararema,SP,Sudeste,2010-01-01,2010-01-01,3518305,nan,Reconhecido,nan,4,2,0,66,417,0,14430,14919,nan,161,91,18604069.987,0.000,0,0.000,2.000,0,24931.748,0,0,0.000,0,0,37397.621,9000.000,0.000,57771845.314,76438244.669,nan,nan,nan,nan,nan,nan,nan,0.000,179508.582,264276.524,12465.874,0.000,349044.465,74795.243,134631.437,0.000,0.000,77288.417,1092010.541,nan,441291.931,2049389.645,0.000,0.000,112192.889,2602874.464,3694885.005,2010,nan,nan,nan,Guararema - SP,1111405.000,nan,nan,nan,nan,nan,nan,Natural,Geológico,Movimento de massa,Deslizamentos,Deslizamentos de solo e ou rocha,11321,25844
2,BA-A-2905909-14110-20100101,Campo Alegre de Lourdes,BA,Nordeste,2010-01-01,2010-01-01,2905909,nan,Reconhecido,nan,0,0,0,0,0,0,0,0,nan,0,0,0.000,0.000,0,0.000,0.000,0,0.000,0,0,0.000,0,0,0.000,0.000,0.000,0.000,0.000,nan,nan,nan,nan,nan,nan,nan,747.952,448771.455,0.000,0.000,0.000,0.000,0.000,249317.475,0.000,0.000,177015.407,875852.290,nan,254303.825,147097.310,0.000,0.000,0.000,401401.135,1277253.425,2010,nan,nan,2914.587,Campo Alegre de Lourdes - BA,107459.000,nan,nan,nan,nan,nan,nan,Natural,Climatológico,Seca,Estiagem,nan,14110,28090
3,SP-A-3513603-12100-20100101,Cunha,SP,Sudeste,2010-01-01,2010-01-01,3513603,nan,Reconhecido,nan,6,0,0,38,492,0,12000,12536,nan,89,9,3340854.165,0.000,0,0.000,0.000,0,0.000,0,0,0.000,0,0,0.000,3.000,400.000,8052097.614,11392951.779,nan,nan,nan,nan,nan,nan,nan,0.000,1246.587,0.000,0.000,0.000,1994539.800,0.000,125656.007,0.000,0.000,0.000,2121442.395,nan,49863.495,1383711.986,0.000,0.000,925466.467,2359041.949,4480484.344,2010,nan,nan,1407.250,Cunha - SP,112225.000,nan,nan,nan,nan,nan,nan,Natural,Hidrológico,Inundações,nan,nan,12100,21866
4,BA-P-2917334-12200-20100101,Iuiu,BA,Nordeste,2010-01-01,2010-01-01,2917334,nan,Reconhecido,nan,0,0,0,0,0,0,0,0,nan,0,0,0.000,0.000,0

In [3]:
df_eventos['Status'] = df_eventos['Status'].map({'Reconhecido': 0, 'Não reconhecido': 1})

In [4]:
df_eventos = df_eventos[["Status", "DH_Descricao", "DM_Descricao", "DA_Descricao"]]

In [5]:
df_DH_Descricao = df_eventos[['Status', 'DH_Descricao']].dropna(subset=['Status', 'DH_Descricao'])

In [6]:
df_DM_Descricao = df_eventos[['Status', 'DM_Descricao']].dropna(subset=['Status', 'DM_Descricao'])

In [7]:
df_DA_Descricao = df_eventos[['Status', 'DA_Descricao']].dropna(subset=['Status', 'DA_Descricao'])

## Modelo para "Descrição de danos humanos"

In [8]:
X_train_humanos, X_test_humanos, y_train_humanos, y_test_humanos = train_test_split(df_DH_Descricao['DH_Descricao'], df_DH_Descricao['Status'], test_size=0.2, random_state=42, stratify=df_DH_Descricao['Status'])

In [9]:
X_train_humanos[0]

KeyError: 0

In [19]:
print('TREINO : ',y_train_humanos.shape,'\n', y_train_humanos.value_counts(normalize=True),'\n')
print('TESTE  : ',y_test_humanos.shape,'\n', y_test_humanos.value_counts(normalize=True))

TREINO :  (13699,) 
 Status
0    0.912548
1    0.087452
Name: proportion, dtype: float64 

TESTE  :  (3425,) 
 Status
0    0.912701
1    0.087299
Name: proportion, dtype: float64


In [20]:
# instancia o transform CountVectorizer
tfidf_vectorizer_humanos = TfidfVectorizer()
# # tokeniza e cria o vocabulário
tfidf_vectorizer_humanos.fit(X_train_humanos)
# # mostra o vocabulário criado
print('Vocabulário: ')
print(tfidf_vectorizer_humanos.vocabulary_)

Vocabulário: 
{'devido': 11239, 'ao': 5571, 'desastre': 10771, 'de': 10366, 'inundação': 15955, 'ocorrida': 19303, 'no': 19002, 'município': 18668, 'faria': 13385, 'lemos': 16950, 'em': 12049, '20': 1169, '02': 53, '2021': 1208, 'foram': 13821, 'contabilizados': 9543, 'número': 19174, '23': 1390, 'pessoas': 20482, 'desabrigadas': 10698, 'que': 21936, 'tiveram': 25318, 'deixar': 10488, 'suas': 24476, 'residências': 22803, 'aumento': 6403, 'abrupto': 4222, 'do': 11611, 'rio': 23069, 'carangola': 7969, 'são': 24794, 'mateus': 17873, 'sendo': 23841, 'levadas': 16986, 'para': 19783, 'abrigos': 4210, 'disponibilizados': 11467, 'pela': 20163, 'prefeitura': 21235, '_x000d_': 4075, '06': 136, 'alojadas': 5137, 'centro': 8411, 'cultural': 10134, 'fica': 13596, 'na': 18771, 'rua': 23252, 'josé': 16532, 'dittz': 11545, 'sn': 24214, 'ponto': 20918, 'referência': 22435, 'parque': 19913, 'exposição': 13208, '13': 572, 'escola': 12593, 'municipal': 18654, 'diogo': 11390, 'vasconcelos': 26278, 'coronel

In [21]:
pipeline_humanos = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2))),
    ('clf', SGDClassifier(loss='log_loss')),
])

classificador_humanos = pipeline_humanos.fit(X_train_humanos, y_train_humanos)

 predicted_humanos = classificador_humanos.predict(X_test_humanos)

In [30]:
print(metrics.classification_report(y_test_humanos, predicted_humanos))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      3126
           1       1.00      0.01      0.01       299

    accuracy                           0.91      3425
   macro avg       0.96      0.50      0.48      3425
weighted avg       0.92      0.91      0.87      3425



### Modelo para descrição de danos materiais

In [31]:
X_train_materiais, X_test_materiais, y_train_materiais, y_test_materiais = train_test_split(df_DM_Descricao['DM_Descricao'], df_DM_Descricao['Status'], test_size=0.2, random_state=42, stratify=df_DM_Descricao['Status'])

In [32]:
X_train_materiais[0]

'Os danos são representados pelas avarias/destruições acentuadas resultantes nas estradas/pontes vicinais do município, em virtude das enxurradas, que resultaram na erosão/deslocamento acentuado do saibro alocado nas estradas, bem como em sérios danos aos sistemas de drenagens/pontes de madeira/concreto, em que algumas foram arrancadas em sua totalidade, além da queda de barreiras e do estreitamento das pistas de rolamento das estradas vicinais, devido ao avanço do leito do rio.\n\nAdemais, houve danos às residências localizadas no entorno do rio, bem como à unidade de saúde/escola/galpão comunitário da Comunidade do Rio Novo, principalmente devido à interrupção do acesso e dos serviços ofertados.'

In [33]:
print('TREINO : ',y_train_materiais.shape,'\n', y_train_materiais.value_counts(normalize=True),'\n')
print('TESTE  : ',y_test_materiais.shape,'\n', y_test_materiais.value_counts(normalize=True))

TREINO :  (9516,) 
 Status
0    0.897226
1    0.102774
Name: proportion, dtype: float64 

TESTE  :  (2379,) 
 Status
0    0.897436
1    0.102564
Name: proportion, dtype: float64


In [34]:
# instancia o transform CountVectorizer
tfidf_vectorizer_materiais = TfidfVectorizer()
# # tokeniza e cria o vocabulário
tfidf_vectorizer_materiais.fit(X_train_materiais)
# # mostra o vocabulário criado
print('Vocabulário: ')
print(tfidf_vectorizer_materiais.vocabulary_)

Vocabulário: 
{'no': 18632, 'bairro': 5037, 'pirulito': 20431, 'nas': 18376, 'ruas': 22982, 'pedro': 19895, 'ii': 14273, 'rua': 22976, 'beira': 5365, 'rio': 22715, 'uma': 25862, 'casa': 6951, 'destruída': 9987, 'parcialmente': 19585, 'danificada': 9207, 'paredes': 19601, 'com': 7920, 'avarias': 4895, 'duas': 10563, 'casas': 6963, 'enxurradas': 11327, 'inundações': 14974, 'que': 21589, 'entraram': 11268, 'água': 27049, 'dentro': 9495, 'danificando': 9218, 'bens': 5453, 'materiais': 17326, 'parte': 19636, 'da': 9137, 'bela': 5377, 'vista': 26615, 'telhado': 24894, 'destruído': 9991, 'deslizamento': 9793, 'de': 9299, 'massa': 17292, 'quintal': 21677, 'outra': 19263, 'centro': 7305, 'um': 25860, 'muro': 18236, 'desabou': 9592, 'na': 18328, 'salvador': 23186, 'outro': 19266, 'residência': 22465, 'duque': 10584, 'caxias': 7145, 'branca': 5897, 'jk': 15564, 'manoel': 16998, 'vital': 26638, 'palmeiras': 19410, 'belarmino': 5379, 'dias': 10136, 'oito': 19019, 'por': 20728, 'motivo': 18069, 'inu

In [35]:
pipeline_materiais = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2))),
    ('clf', SGDClassifier(loss='log_loss')),
])

classificador_materiais = pipeline_materiais.fit(X_train_materiais, y_train_materiais)

predicted_materiais = classificador_materiais.predict(X_test_materiais)

In [36]:
print(metrics.classification_report(y_test_materiais, predicted_materiais))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      2135
           1       0.00      0.00      0.00       244

    accuracy                           0.90      2379
   macro avg       0.45      0.50      0.47      2379
weighted avg       0.81      0.90      0.85      2379



#### Modelo para descrição de danos ambientais

In [41]:
X_train_ambiental, X_test_ambiental, y_train_ambiental, y_test_ambiental = train_test_split(df_DA_Descricao['DA_Descricao'], df_DA_Descricao['Status'], test_size=0.2, random_state=42, stratify=df_DA_Descricao['Status'])

In [46]:
X_train_ambiental[13282]

'Não há registros de danos ambientais.'

In [ ]:
print('TREINO : ',y_train_ambiental.shape,'\n', y_train_materiais.value_counts(normalize=True),'\n')
print('TESTE  : ',y_test_ambiental.shape,'\n', y_test_materiais.value_counts(normalize=True))

In [ ]:
# instancia o transform CountVectorizer
tfidf_vectorizer_materiais = TfidfVectorizer()
# # tokeniza e cria o vocabulário
tfidf_vectorizer_materiais.fit(X_train_materiais)
# # mostra o vocabulário criado
print('Vocabulário: ')
print(tfidf_vectorizer_materiais.vocabulary_)

In [ ]:
pipeline_materiais = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2))),
    ('clf', SGDClassifier(loss='log_loss')),
])

classificador_materiais = pipeline_materiais.fit(X_train_materiais, y_train_materiais)

predicted_materiais = classificador_materiais.predict(X_test_materiais)

In [ ]:
print(metrics.classification_report(y_test_materiais, predicted_materiais))